In [1]:
import os
import json
import pandas as pd
from src.gender import gender
from src.vycisti import vycisti
pd.set_option('display.max_columns', None)
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())
with open(os.path.join("data_fixes", "seznamy.json"), encoding="utf-8") as seznamy:
    seznamy = json.loads(seznamy.read())
with open(os.path.join("data_fixes", "zeny.json"), encoding="utf-8") as zeny:
    zeny = json.loads(zeny.read())
with open(os.path.join("data_fixes", "nelide.json"), encoding="utf-8") as nelide:
    nelide = json.loads(nelide.read())
df = vycisti(pd.read_json(os.path.join("data", "filmy.json")),opravy)

2023/05/29 12:47:50 z 4555 řádků odstraněno 125. Aktuální dataframe má 4430 řádků.


In [2]:
df["Hrají"]

0                                                    None
1                       [Josef Šváb-Malostranský (55291)]
2       [Josef Šváb-Malostranský (55291), Ferdinand Gý...
3                                                    None
4                                                    None
                              ...                        
4550                                                 None
4551                                                 None
4552    [Ben Foster (87489), Sophie Lowe (41037), Til ...
4553    [Patrik Děrgel (83665), Tereza Vilišová (1699)...
4554    [Pavel Šimčík (8265), Iva Janžurová (1049), Iv...
Name: Hrají, Length: 4430, dtype: object

In [3]:
df["Hlavní role"] = df["Hrají"].dropna().apply(lambda x: x[:3])

In [4]:
df["Vedlejší role"] = df["Hrají"].dropna().apply(lambda x: x[3:])

## Gender hlavních rolí

In [5]:
def gender_hlavnich_roli(x):

    global zeny
    global nelide
    
    if not isinstance(x, list):
        return x
    
    muz_item = []
    zena_item = []
    
    for item in x:
        
        item_stripped = item.split("(")[0].strip()
        gender_result = gender(item_stripped, zeny, nelide)
        if gender_result == "muž":
            muz_item.append(item)
        elif gender_result == "žena":
            zena_item.append(item)
            
    return [muz_item, zena_item]

In [6]:
df["Muži v hlavních rolích"] = df["Hlavní role"].dropna().apply(lambda x: gender_hlavnich_roli(x)[0])
df["Ženy v hlavních rolích"] = df["Hlavní role"].dropna().apply(lambda x: gender_hlavnich_roli(x)[1])

In [7]:
df.explode("Muži v hlavních rolích").explode("Ženy v hlavních rolích").groupby(["Muži v hlavních rolích","Ženy v hlavních rolích"])["Film"].nunique().nlargest(10)

Muži v hlavních rolích    Ženy v hlavních rolích     
Karel Lamač (37847)       Anny Ondráková (127280)        17
Theodor Pištěk (37366)    Antonie Nedošinská (127295)    12
                          Anny Ondráková (127280)         9
Otto Šimánek (35353)      Veronika Renčová (128659)       7
Theodor Pištěk (37366)    Máňa Ženíšková (126674)         7
Jan Werich (1287)         Veronika Renčová (128659)       6
Karel Roden nejml. (494)  Ivana Chýlková (4684)           6
Hugo Haas (37289)         Adina Mandlová (55611)          5
                          Antonie Nedošinská (127295)     5
Jaromír Hanzlík (14425)   Daniela Kolářová (1847)         5
Name: Film, dtype: int64

In [8]:
df[df["Copyright"] > 1945].explode("Muži v hlavních rolích").explode("Ženy v hlavních rolích").groupby(["Muži v hlavních rolích","Ženy v hlavních rolích"])["Film"].nunique().nlargest(35)

Muži v hlavních rolích      Ženy v hlavních rolích     
Otto Šimánek (35353)        Veronika Renčová (128659)      7
Jan Werich (1287)           Veronika Renčová (128659)      6
Karel Roden nejml. (494)    Ivana Chýlková (4684)          6
Jaromír Hanzlík (14425)     Daniela Kolářová (1847)        5
Vlastimil Brodský (833)     Jana Brejchová (7710)          5
Boleslav Polívka (1556)     Eva Holubová (5948)            4
Gustáv Valach (10715)       Jana Brejchová (7710)          4
Jiří Schmitzer (2936)       Helena Vitovská (34062)        4
Jiří Sovák (5288)           Iva Janžurová (1049)           4
Josef Abrhám (2274)         Libuše Šafránková (2272)       4
Tomáš Holý (27091)          Jana Brejchová (7710)          4
Jan Kačer (1165)            Jana Brejchová (7710)          3
Jiří Bartoška (827)         Ivana Chýlková (4684)          3
Jiří Sovák (5288)           Jana Hlaváčová (11370)         3
Josef Šebánek (69908)       Marie Motlová (35339)          3
Karel Höger (38146)         D

## První muž v titulcích + první žena v titulcích

In [9]:
def hlavni_hvezdy(x):

    if not isinstance(x, list):
        return x
    
    muz_item = None
    zena_item = None
    
    for item in x:
        
        item_stripped = item.split("(")[0].strip()
        gender_result = gender(item_stripped, zeny, nelide)
        if gender_result == "muž" and not muz_item:
            muz_item = item
        elif gender_result == "žena" and not zena_item:
            zena_item = item
        
        if muz_item and zena_item:
            break
    
    return [muz_item, zena_item]

In [10]:
pokus = ["Igor Chmela (26627)", "Jana Plodková (24916)"]

In [11]:
hlavni_hvezdy(pokus)

['Igor Chmela (26627)', 'Jana Plodková (24916)']

In [12]:
df["Muž v hlavní roli"] = df["Hrají"].dropna().apply(lambda x: hlavni_hvezdy(x)[0])
df["Žena v hlavní roli"] = df["Hrají"].dropna().apply(lambda x: hlavni_hvezdy(x)[1])

In [13]:
df.groupby(["Muž v hlavní roli","Žena v hlavní roli"])["Film"].nunique().nlargest(10)

Muž v hlavní roli        Žena v hlavní roli         
Theodor Pištěk (37366)   Antonie Nedošinská (127295)    12
Karel Lamač (37847)      Anny Ondráková (127280)        10
Otto Šimánek (35353)     Veronika Renčová (128659)       7
Theodor Pištěk (37366)   Anny Ondráková (127280)         5
Zdeněk Štěpánek (87457)  Vlasta Matulová (998)           5
Boleslav Polívka (1556)  Eva Holubová (5948)             4
Hugo Haas (37289)        Antonie Nedošinská (127295)     4
Jindřich Plachta (5829)  Zdeňka Baldová (28798)          4
Jiří Schmitzer (2936)    Helena Vitovská (34062)         4
Karel Noll (122320)      Antonie Nedošinská (127295)     4
Name: Film, dtype: int64

In [14]:
df[df["Copyright"] > 1945].groupby(["Muž v hlavní roli","Žena v hlavní roli"])["Film"].nunique().nlargest(20)

Muž v hlavní roli           Žena v hlavní roli         
Otto Šimánek (35353)        Veronika Renčová (128659)      7
Boleslav Polívka (1556)     Eva Holubová (5948)            4
Jiří Schmitzer (2936)       Helena Vitovská (34062)        4
Vlastimil Brodský (833)     Jana Brejchová (7710)          4
Zdeněk Štěpánek (87457)     Vlasta Matulová (998)          4
Gustáv Valach (10715)       Jana Brejchová (7710)          3
Jan Kačer (1165)            Jana Brejchová (7710)          3
Jiří Sovák (5288)           Jiřina Bohdalová (2878)        3
Josef Abrhám (2274)         Libuše Šafránková (2272)       3
Josef Šebánek (69908)       Marie Motlová (35339)          3
Karel Heřmánek (6274)       Jana Synková (12616)           3
Karel Höger (38146)         Dana Medřická (64028)          3
                            Slávka Budínová (8606)         3
Martin Finger (19835)       Anna Geislerová (5433)         3
Ondřej Vetchý (836)         Anna Geislerová (5433)         3
Otomar Krejča (7597)        A

In [15]:
df[df["Copyright"] > 1989].groupby(["Muž v hlavní roli","Žena v hlavní roli"])["Film"].nunique().nlargest(20)

Muž v hlavní roli           Žena v hlavní roli       
Boleslav Polívka (1556)     Eva Holubová (5948)          4
Karel Heřmánek (6274)       Jana Synková (12616)         3
Martin Finger (19835)       Anna Geislerová (5433)       3
Ondřej Vetchý (836)         Anna Geislerová (5433)       3
Václav Vydra nejml. (2666)  Jana Paulová (8262)          3
Boleslav Polívka (1556)     Zlata Adamovská (18668)      2
                            Zuzana Bydžovská (4683)      2
Hynek Čermák (3370)         Vlastina Svatková (81246)    2
Jakub Štáfek (82135)        Šárka Vaculíková (16033)     2
Jaromír Dulava (1837)       Marie Mazurová (4266)        2
Jiří Bartoška (827)         Ivana Chýlková (4684)        2
Jiří Langmajer (2981)       Anna Polívková (15610)       2
Jiří Schmitzer (2936)       Helena Vitovská (34062)      2
Josef Abrhám (2274)         Libuše Šafránková (2272)     2
Karel Roden nejml. (494)    Anna Geislerová (5433)       2
                            Chantal Poullain (1573)      2
  

In [16]:
df[df["Copyright"] > 1998].groupby(["Muž v hlavní roli","Žena v hlavní roli"])["Film"].nunique().nlargest(20)

Muž v hlavní roli           Žena v hlavní roli       
Boleslav Polívka (1556)     Eva Holubová (5948)          4
Martin Finger (19835)       Anna Geislerová (5433)       3
Ondřej Vetchý (836)         Anna Geislerová (5433)       3
Václav Vydra nejml. (2666)  Jana Paulová (8262)          3
Boleslav Polívka (1556)     Zlata Adamovská (18668)      2
                            Zuzana Bydžovská (4683)      2
Hynek Čermák (3370)         Vlastina Svatková (81246)    2
Jakub Štáfek (82135)        Šárka Vaculíková (16033)     2
Jiří Langmajer (2981)       Anna Polívková (15610)       2
Josef Abrhám (2274)         Libuše Šafránková (2272)     2
Karel Roden nejml. (494)    Anna Geislerová (5433)       2
                            Lenka Vlasáková (1912)       2
Kryštof Hádek (1830)        Tereza Voříšková (26752)     2
Martin Myšička (13007)      Jana Plodková (24916)        2
Martin Pechlát (9426)       Gabriela Míčová (9758)       2
Miroslav Šimůnek (3860)     Michaela Kuklová (3313)      2
Ol

In [17]:
df[(df["Muž v hlavní roli"] == "Boleslav Polívka (1556)") & (df["Žena v hlavní roli"] == "Eva Holubová (5948)")]["Film"].drop_duplicates().to_list()

['Pupendo (18262)',
 'Skřítek (27850)',
 'Manžel na hodinu (401145)',
 'Gump – pes, který naučil lidi žít (402147)']

In [18]:
df.groupby("Žena v hlavní roli")["Film"].nunique().nlargest(20)

Žena v hlavní roli
Antonie Nedošinská (127295)    40
Jana Brejchová (7710)          35
Iva Janžurová (1049)           30
Anny Ondráková (127280)        27
Adina Mandlová (55611)         26
Hana Vítová (126418)           26
Anna Geislerová (5433)         24
Jiřina Bohdalová (2878)        24
Ivana Chýlková (4684)          21
Jiřina Štěpničková (29765)     21
Zdeňka Baldová (28798)         20
Lída Baarová (5202)            19
Jiřina Šejbalová (125559)      18
Libuše Šafránková (2272)       17
Slávka Budínová (8606)         17
Dana Medřická (64028)          16
Květa Fialová (5338)           16
Věra Ferbasová (33991)         16
Andula Sedláčková (126987)     15
Jana Hlaváčová (11370)         15
Name: Film, dtype: int64

In [19]:
df.explode("Režie").groupby(["Režie","Žena v hlavní roli"])["Film"].nunique().nlargest(20)

Režie                       Žena v hlavní roli         
Karel Lamač (37847)         Anny Ondráková (127280)        14
Max Urban (95056)           Andula Sedláčková (126987)     10
Vladimír Slavínský (31967)  Antonie Nedošinská (127295)    10
                            Věra Ferbasová (33991)          9
Jindřich Polák (7891)       Veronika Renčová (128659)       7
Martin Frič (20324)         Adina Mandlová (55611)          7
Jan Hřebejk (4831)          Anna Geislerová (5433)          6
Svatopluk Innemann (33860)  Zdena Kavková (127457)          6
Martin Frič (20324)         Běla Jurdová (27291)            5
                            Zdeňka Baldová (28798)          5
Miroslav Cikán (31969)      Hana Vítová (126418)            5
Oldřich Kmínek (37986)      Anita Janová (128156)           5
Otakar Vávra (9507)         Vlasta Matulová (998)           5
Evald Schorm (7895)         Jana Brejchová (7710)           4
Jiří Slavíček (31972)       Hana Vítová (126418)            4
Martin Frič (2

In [20]:
df.explode("Režie").groupby(["Režie","Muž v hlavní roli"])["Film"].nunique().nlargest(20)

Režie                       Muž v hlavní roli         
Jindřich Polák (7891)       Otto Šimánek (35353)          16
Václav Binovec (37467)      Suzanne Marwille (127373)     15
Martin Frič (20324)         Vlasta Burian (11992)         10
Karel Lamač (37847)         Karel Lamač (37847)            8
                            Vlasta Burian (11992)          8
Dušan Klein (2568)          Pavel Kříž (2990)              7
Martin Frič (20324)         Hugo Haas (37289)              7
Otakar Vávra (9507)         Zdeněk Štěpánek (87457)        7
Martin Frič (20324)         František Smolík (126955)      6
                            Oldřich Nový (12009)           6
Vladimír Slavínský (31967)  Theodor Pištěk (37366)         6
Karel Lamač (37847)         Theodor Pištěk (37366)         5
Martin Frič (20324)         Zdeněk Štěpánek (87457)        5
Miroslav Cikán (31969)      Jindřich Plachta (5829)        5
Otakar Vávra (9507)         Karel Höger (38146)            5
Svatopluk Innemann (33860)  Th

## Hvězdnost herectva

In [21]:
hvezdnost = pd.Series(df.explode("Hlavní role").groupby("Hlavní role")["Film"].nunique() / df.explode("Hrají").groupby("Hrají")["Film"].nunique(), name="Hvězdnost")

In [22]:
hvezdnost

2 Amiros                                             NaN
2 Arotty                                             NaN
2 Bertis                                             NaN
2 Olympos                                            NaN
2 Poldis                                             NaN
                                                    ... 
žáci královéhradeckých škol                          NaN
žáci lyžařských sportovních základen v Krkonoších    NaN
žáci osmiletých středních škol v Gottwaldově         1.0
žáci pražských škol                                  NaN
žáci základní školy Praha 4                          NaN
Name: Hvězdnost, Length: 37317, dtype: float64

In [23]:
obsazovanost = pd.Series(df.explode("Hrají").groupby("Hrají")["Film"].nunique(), name="Obsazovanost")

In [24]:
obsazovanost

Hrají
2 Amiros                                             1
2 Arotty                                             1
2 Bertis                                             1
2 Olympos                                            1
2 Poldis                                             1
                                                    ..
žáci královéhradeckých škol                          1
žáci lyžařských sportovních základen v Krkonoších    1
žáci osmiletých středních škol v Gottwaldově         1
žáci pražských škol                                  1
žáci základní školy Praha 4                          1
Name: Obsazovanost, Length: 37317, dtype: int64

In [25]:
hvezdy = pd.DataFrame([hvezdnost, obsazovanost]).transpose()

In [26]:
hvezdy

,Hvězdnost,Obsazovanost
2 Amiros,NaN,1.0
2 Arotty,NaN,1.0
2 Bertis,NaN,1.0
2 Olympos,NaN,1.0
2 Poldis,NaN,1.0
...,...,...
žáci královéhradeckých škol,NaN,1.0
žáci lyžařských sportovních základen v Krkonoších,NaN,1.0
žáci osmiletých středních škol v Gottwaldově,1.0,1.0
žáci pražských škol,NaN,1.0


In [27]:
q = 30

In [28]:
hvezdy[hvezdy["Obsazovanost"] >= q].sort_values(by="Hvězdnost", ascending=False).head(60)

,Hvězdnost,Obsazovanost
Vlasta Burian (11992),0.970588,34.0
Suzanne Marwille (127373),0.900000,40.0
Hugo Haas (37289),0.766667,30.0
Anny Ondráková (127280),0.735294,34.0
Anna Geislerová (5433),0.710526,38.0
Karel Lamač (37847),0.698413,63.0
Karel Höger (38146),0.691176,68.0
Karel Roden nejml. (494),0.666667,54.0
Lída Baarová (5202),0.656250,32.0
Jana Brejchová (7710),0.650000,60.0


In [29]:
burian = df.explode("Vedlejší role")
burian[burian["Vedlejší role"] == "Vlasta Burian (11992)"]

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,Originální název,Český název,Anglický název,Souběžný název,Pracovní název,Hrají,Postavy,Pomocná režie,Asistent režie,Původní filmový námět,Scénář,Dramaturg,Kamera,Asistent kamery,Architekt,Návrhy kostýmů,Masky,Střih,Zvuk,Producent,Vedoucí výroby,Zástupce vedoucího výroby,Odborný poradce,Hudba,Choreografie,Lokace,Distribuční slogan,Výrobce,S podporou,Nositelé copyrightu,Distribuce,Délka,Distribuční nosič,Poměr stran,Barva,Zvukový,Zvukový systém/formát,Verze,Mluveno,Podtitulky,Úvodní/závěrečné titulky,Anotace,Obsah,Technický scénář,Výtvarník,Animace,Triky,Postprodukce,Návrhy titulků,Pedagogické vedení,Použitá hudba,Studentský film,Původní metráž,Mezititulky,Poznámka,Mluví,Vizuální efekty,Vedoucí produkce,Předloha,Kaskadéři,Text písně,Zpívá,pásmo,Části,Produkce,Výtvarná spolupráce,Nahrál,Partner,Animační technika,Výprava,Spolupráce,Komentář,Autor komentáře,Druhá kamera,Zvláštní efekty,Koproducent,Výběr hudby,Hudební dramaturgie,Písně,"Hudba, text a zpěv písně",Produkční data,Koordinátor kaskadérů,Výkonná produkce,Zástupce vedoucího produkce,Hudební režie,Účinkují,Dirigent,Ateliéry,Režie české verze,Výrobce české verze,Tančí,České podtitulky,Výrobce českých podtitulků,Asistent vedoucího produkce,Natáčecí formát,Dialogy české verze,Autor projektu,Stav filmu,Souběžný anglický název,Assistent architekta,Asistent zvuku,Podnázev,Asistent vedoucího výroby,Skript,Kostýmy,Asistent střihu,povídkový film,Povídky,Hudba k písni,Prodej práv,Podnázev anglický,Výtvarník pozadí,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,cyklus,Začátek výroby,Konec výroby,Hlavní role,Vedlejší role,Muži v hlavních rolích,Ženy v hlavních rolích,Muž v hlavní roli,Žena v hlavní roli
1494,Zaostřit prosím! (396247),Československo,1956,1956,premiéra 25. 12. 1956 /přístupný mládeži/ (kin...,77.0,[Martin Frič (20324)],film,"[komedie, satira]","[hraný, distribuční]",Zaostřit prosím!,Zaostřit prosím!,"Focus, Please!",None,None,"[Jan Pivec (127312), Josef Kemr (4897), Zdeňka...","[kritik Pošahal, účetní a vedoucí stavby Jan J...",[Jiří Jungwirth (83597)],[Marie Hejzlarová (69926)],[Jiří Marek (4851)],[Jiří Marek (4851)],None,[Václav Pazderník (127339)],"[Jaroslav Kaiser (92572), Stanislav Kautský (1...",[Karel Škvor (81834)],None,[Josef Mann (18269)],[Jan Kohout (126135)],[Josef Vlček (126072)],None,[Ladislav Terš (3631)],[Rudolf Wolf (126989)],None,[31512],None,[],Český film. Satira zaostřená na nás všechny.,[Studio hraných filmů (124796)],None,[Národní filmový archiv (92324)],[Rozdělovna filmů Československého státního fi...,dlouhometrážní,"[16mm, 35mm]",1.37,černobílý,zvukový,mono,česká,česky,bez titulků,české,None,Při promítání v zaplněném kině se přetrhl film...,[Martin Frič (20324)],None,None,None,None,None,None,None,None,2203.0,None,None,None,None,None,"[4851, Jiří Marek (Jak účetní k štěstí přišel ...",None,None,None,None,None,None,None,"[15006, 5613]",None,None,[Jan Janda (131549)],None,None,None,None,None,None,None,None,[Čeká nás fůra práce],[Ludvík Podéšť (31512)],promítání povoleno 4. 12. 1956,None,None,None,None,None,None,Hostivař,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,[Aša Teršová (6283)],None,None,None,None,"Národní filmový archiv, Obchodní oddělení",None,None,None,None,None,None,None,None,None,None,None,1956,1956,"[Jan Pivec (127312), Josef Kemr (4897), Zdeňka...",Vlasta Burian (11992),"[Jan Pivec (127312), Josef Kemr (4897)]",[Zdeňka Baldová (28798)],Jan Pivec (127312),Zdeňka Baldová (28798)


In [30]:
hvezdy[(hvezdy["Obsazovanost"] >= 30) & (hvezdy["Hvězdnost"].isnull())].sort_values(by="Obsazovanost", ascending=False).head(10)

,Hvězdnost,Obsazovanost
Karel Hovorka (76534),NaN,135.0
Vladimír Navrátil (12670),NaN,126.0
Jiřina Bílá (3629),NaN,115.0
Jan Kuželka (1445),NaN,115.0
Zdeněk Srstka (7960),NaN,111.0
Karel Bělohradský (6370),NaN,101.0
Zdeněk Skalický (126510),NaN,93.0
Jan Cmíral (125562),NaN,92.0
Emanuel Hříbal (127758),NaN,92.0
Antonín Jirsa (129711),NaN,85.0


In [31]:
hvezdy[(hvezdy["Obsazovanost"] >= 30) & (hvezdy["Hvězdnost"].isnull())].sort_values(by="Obsazovanost", ascending=False).shape

(116, 2)

In [32]:
hvezdy[(hvezdy["Obsazovanost"] >= 30) & (~hvezdy["Hvězdnost"].isnull())].sort_values(by="Hvězdnost", ascending=True).head(10)

,Hvězdnost,Obsazovanost
Jaroslav Tomsa (2576),0.006579,152.0
Karel Engel (14390),0.009174,109.0
Josef Oliak (128024),0.010870,92.0
Alois Dvorský (127491),0.011976,167.0
Karel Pavlík (126439),0.012658,79.0
Miloslav Homola (18975),0.012821,78.0
Vlastimila Vlková (27109),0.013333,75.0
Viktor Maurer (78076),0.013889,72.0
Karel Augusta (5058),0.015625,128.0
F. X. Mlejnek (128287),0.015625,128.0
